# 기본 패키지 import

In [201]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## selenium import 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

### 환경변수용 

import os
import requests
from dotenv import load_dotenv
load_dotenv()
from geopy import distance



In [195]:
from geopy import distance

# 스타벅스 데이터 추출하기 
--- 
스타벅스 홈페이지로부터 데이터를 추출해서 데이터프레임화 한다.

스타벅스 데이터 추출하기



In [140]:
## 화면에 따른 element 나 dom 변경을 방지하기위해 화면 사이즈를 특정 사이즈로 고정

options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
driver = webdriver.Chrome(options=options)


크롤링 페이지 : https://www.starbucks.co.kr/store/store_map.do?disp=locale


In [141]:
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

로딩 마커 확인 
- xpath : /html/body/div[@class='loading_dim']
- 해당 요소가 가려질때까지 대기

In [142]:


##

loading_check=driver.find_element(By.XPATH, '/html/body/div[@class="loading_dimm"]')
while loading_check.is_displayed():
    print(f' loading check... : style: {loading_check.get_attribute("style")}')
    time.sleep(0.1)
    loading_check=driver.find_element(By.XPATH, '/html/body/div[@class="loading_dimm"]')

 loading check... : style: z-index: 20000; display: block; opacity: 0.136146;
 loading check... : style: z-index: 20000; display: block; opacity: 0.441661;
 loading check... : style: z-index: 20000; display: block; opacity: 0.630543;
 loading check... : style: z-index: 20000; display: block; opacity: 0.697882;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 200

지역 화면 활성화 대기

In [143]:
local_xpath='//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/header[@class="loca_search"]'

while True:
    try:
        local=driver.find_element(By.XPATH, local_xpath)
        break
    except:
        print('retrying...')
        time.sleep(0.1)


목차 화면대기

In [144]:
area_select_xpath='//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a'

while True:
    try:
        area_select=driver.find_element(By.XPATH, area_select_xpath)
        break
    except:
        print('retrying...')
        time.sleep(1)

area_select.click()

all_selct_xpath='//*[@id="mCSB_2_container"]/ul/li[1]/a'
while True:
    try:
        all_select=driver.find_element(By.XPATH, all_selct_xpath)
        break
    except:
        print('retrying...')
        time.sleep(1)

all_select.click()


retrying...


In [145]:
items='//*[@id="mCSB_3_container"]/ul/li'

while True:
    try:
        item_list=driver.find_elements(By.XPATH, items)
        if len(item_list)>0:
            break
    except:
        print('retrying...')
        time.sleep(1)
len(item_list)

613

In [146]:
item_list[0].get_attribute('innerText').split('\n')
## 이름 : 0 , 주소 : 2, 전화번호 : 3

['역삼아레나빌딩', '', '서울특별시 강남구 언주로 425 (역삼동)', '1522-3232', '', '리저브 매장 2번']

In [147]:
rets=[]
for item in item_list:
    ret={}
    item_ele=item.get_attribute('innerText').split('\n')
    ret['name']=item_ele[0]
    ret['address']=item_ele[2]
    #ret['phone']=item_ele[3]
    ret['type']='starbucks'
    ret['lat']=item.get_attribute('data-lat')
    ret['lng']=item.get_attribute('data-long')
    rets.append(ret)

df=pd.DataFrame(rets)
df.head()

,name,address,type,lat,lng
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),starbucks,37.501087,127.043069
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),starbucks,37.510178,127.022223
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),starbucks,37.5139309,127.0206057
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),starbucks,37.499517,127.031495
4,대치재경빌딩,서울특별시 강남구 남부순환로 2947 (대치동),starbucks,37.494668,127.062583


# 이디아 커피 정보추출
---
이디아 홈페이지로부터 데이터를 추출한다.

In [148]:
url_2='https://www.ediya.com/contents/find_store.html'
driver.get(url_2)

input_xpath='//*[@id="storename"]'
input_box=driver.find_element(By.XPATH, input_xpath)
while input_box.is_displayed():
    try:
        loading_check=driver.find_element(By.XPATH, input_xpath)
        break
    except:
        time.sleep(0.1)
    


In [149]:
input_xpath='//*[@id="keyword"]'
ret_xpath='//*[@id="placesList"]/li'


action='//*[@id="contentWrap"]/div[3]/div/div[1]/ul/li[2]/a'
while True:
    try:
        action_btn=driver.find_element(By.XPATH, action)
        if action_btn.get_attribute('class')=='on':
            break
        else:
            action_btn.click()
    except:
        print('retrying...')
        time.sleep(1)


In [150]:
import re

rets_ediya=[]
input_box=driver.find_element(By.XPATH, input_xpath)
seoul_gu = ['강남구', '강동구', '강북구', '강서구',
             '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']

#rets_ediya=[]
for gu in seoul_gu:
    input_box.clear()
    input_box.send_keys(f"서울 {gu}")
    input_box.send_keys(Keys.RETURN)
    time.sleep(2)
    while True:
        try:
            store_list=driver.find_elements(By.XPATH, ret_xpath)
            if len(store_list)>0:
                break
        except:
            print('retrying...')
            time.sleep(5)
    for store in store_list:
        ret={}
        ret['name']=store.get_attribute('innerText').split('\n')[0]
        ret['address']=store.get_attribute('innerText').split('\n')[1]
        ## latlng 데이터가 안정적이지 않아서 주소로부터 geocoding을 통해 latlng을 가져옴       
        ret['type']='ediya'
        rets_ediya.append(ret)

## update df


에러발생한 케이스에 대해서 예외처리 (geocoding)

In [178]:
## geocoding url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address}'
## headers
api_key=os.getenv('KAKAO_API_KEY')
baseurl='https://dapi.kakao.com/v2/local/search/address.json'
headers = {
    "Authorization": f"KakaoAK {api_key}"
}

def get_latlng(address):
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address}'
    response = requests.get(url, headers=headers)
    data = response.json()
    if data['meta']['total_count'] == 0:
        print(f'No result for {address}')
        return None, None
    return float(data['documents'][0]['y']), float(data['documents'][0]['x'])


rets_processed=[]
for ret in rets_ediya:
    lat, lng = get_latlng(ret['address'])
    ret['lat'] = lat
    ret['lng'] = lng
    if lat is None:
        continue
    rets_processed.append(ret)

df_ediya=pd.DataFrame(rets_processed) ## error 케이스 무시

No result for 서울 강북구 도봉로 207 -3(미아동)
No result for 서울 노원구 한글비석로 409 (상계동) 1~2층
No result for 서울 동작구 사당로16가길 96 (사당동) 1,2층
No result for 서울 마포구 백범로 100 (대흥동)
No result for 서울 영등포구 영등포로35길 19 (영등포동6가)
No result for 서울 은평구 서오릉로 195 (구산동)


In [186]:
df_all=pd.concat([df, df_ediya]).to_csv('starbucks_ediya.csv', index=False)

# 분석
---
### 가설설정
 1. 스타벅스 특정 거리안에 이디아 커피 매장이 있다.
 2. 특정 거리 안에 스타벅스 커피가 없는 이디아 커피 매장이 있다.

In [184]:
## 상관관계 보기

## 스타벅스 주변에 이디야 매장이 많을수록 스타벅스 매장이 많을까?
## 스타벅스 주변 100, 200, 300m 반경에 이디야 매장이 몇개나 있는지 확인


In [185]:
df

,name,address,type,lat,lng
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),starbucks,37.501087,127.043069
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),starbucks,37.510178,127.022223
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),starbucks,37.5139309,127.0206057
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),starbucks,37.499517,127.031495
4,대치재경빌딩,서울특별시 강남구 남부순환로 2947 (대치동),starbucks,37.494668,127.062583
...,...,...,...,...,...
608,사가정역,서울특별시 중랑구 면목로 310,starbucks,37.579594,127.087966
609,상봉역,서울특별시 중랑구 망우로 307 (상봉동),starbucks,37.59689,127.08647
610,묵동,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",starbucks,37.615368,127.076633
611,양원역,서울특별시 중랑구 양원역로10길 3 (망우동),starbucks,37.6066536267232,127.106359790053


In [204]:
rets_count_100m=[]
rets_count_200m=[]
rets_count_500m=[]
rets_count_1000m=[]

for i in df.iloc:
    count_100m=0
    count_200m=0
    count_500m=0
    count_1000m=0
    for j in df_ediya.iloc:
        distance_from_store=distance.distance((i['lat'], i['lng']), (j['lat'], j['lng'])).m
        if distance_from_store<100:
            count_100m+=1
            count_200m+=1
            count_500m+=1
            count_1000m+=1
        elif distance_from_store<200:
            count_200m+=1
            count_500m+=1
            count_1000m+=1
        elif distance_from_store<500:
            count_500m+=1
            count_1000m+=1
        elif distance_from_store<1000:
            count_1000m+=1
        else:
            continue
    rets_count_100m.append(count_100m)
    rets_count_200m.append(count_200m)
    rets_count_500m.append(count_500m)
    rets_count_1000m.append(count_1000m)
    
    ## 

In [208]:
df_distance=pd.DataFrame(
    {
        'name':df['name'],
        'count_100m':rets_count_100m,
        'count_200m':rets_count_200m,
        'count_500m':rets_count_500m,
        'count_1000m':rets_count_1000m
    }
)

df_starbucks=pd.concat([df, df_distance], axis=1)

In [240]:
print("스타벅스 통계 (근처 이디아 매장이 한개이상 있는 경우)")
print(f"100m\t 안에 이디야 매장이 있는 비율 : {sum(df_starbucks['count_100m']>0)}개, {(sum(df_starbucks['count_100m']>0)/len(df_starbucks)*100):2.2f}%") ## 100m 반경에 이디야 매장이 있는 스타벅스 매장의 수
print(f"200m\t 안에 이디야 매장이 있는 비율 : {sum(df_starbucks['count_200m']>0)}개, {(sum(df_starbucks['count_200m']>0)/len(df_starbucks)*100):2.2f}%") ## 100m 반경에 이디야 매장이 있는 스타벅스 매장의 수
print(f"500m\t 안에 이디야 매장이 있는 비율 : {sum(df_starbucks['count_500m']>0)}개, {(sum(df_starbucks['count_500m']>0)/len(df_starbucks)*100):2.2f}%") ## 100m 반경에 이디야 매장이 있는 스타벅스 매장의 수
print(f"1000m\t 안에 이디야 매장이 있는 비율 : {sum(df_starbucks['count_1000m']>0)}개, {(sum(df_starbucks['count_1000m']>0)/len(df_starbucks)*100):2.2f}%") ## 100m 반경에 이디야 매장이 있는 스타벅스 매장의 수

스타벅스 통계 (근처 이디아 매장이 한개이상 있는 경우)
100m	 안에 이디야 매장이 있는 비율 : 132개, 21.53%
200m	 안에 이디야 매장이 있는 비율 : 325개, 53.02%
500m	 안에 이디야 매장이 있는 비율 : 563개, 91.84%
1000m	 안에 이디야 매장이 있는 비율 : 607개, 99.02%


반례. 이디야가 많아서 스타벅스가 없는데에도 있는 경우? (역)  
즉 이디야 주변에 스타벅스가 없는 예시가 있는가?

In [242]:
rets_count_100m=[]
rets_count_200m=[]
rets_count_500m=[]
rets_count_1000m=[]

for i in df_ediya.iloc:
    count_100m=0
    count_200m=0
    count_500m=0
    count_1000m=0
    for j in df.iloc:
        distance_from_store=distance.distance((i['lat'], i['lng']), (j['lat'], j['lng'])).m
        if distance_from_store<100:
            count_100m+=1
            count_200m+=1
            count_500m+=1
            count_1000m+=1
        elif distance_from_store<200:
            count_200m+=1
            count_500m+=1
            count_1000m+=1
        elif distance_from_store<500:
            count_500m+=1
            count_1000m+=1
        elif distance_from_store<1000:
            count_1000m+=1
        else:
            continue
    rets_count_100m.append(count_100m)
    rets_count_200m.append(count_200m)
    rets_count_500m.append(count_500m)
    rets_count_1000m.append(count_1000m)
    
    ## 

df_distance=pd.DataFrame(
    {
        'name':df_ediya['name'],
        'count_100m':rets_count_100m,
        'count_200m':rets_count_200m,
        'count_500m':rets_count_500m,
        'count_1000m':rets_count_1000m
    }
)

df_ediya_w_distance=pd.concat([df_ediya, df_distance], axis=1)

In [251]:
print("이디야 통계 (근처 스타벅스 매장이 없는 경우)")
print(f"100m\t 안에 스타벅스 매장이 없는 비율 : \t{sum(df_ediya_w_distance['count_100m']==0)}개, \t{(sum(df_ediya_w_distance['count_100m']==0)/len(df_ediya_w_distance)*100):2.2f}%") ## 100m 반경에 이디야 매장이 있는 스타벅스 매장의 수
print(f"200m\t 안에 스타벅스 매장이 없는 비율 : \t{sum(df_ediya_w_distance['count_200m']==0)}개, \t{(sum(df_ediya_w_distance['count_200m']==0)/len(df_ediya_w_distance)*100):2.2f}%") ## 100m 반경에 이디야 매장이 있는 스타벅스 매장의 수
print(f"500m\t 안에 스타벅스 매장이 없는 비율 : \t{sum(df_ediya_w_distance['count_500m']==0)}개, \t{(sum(df_ediya_w_distance['count_500m']==0)/len(df_ediya_w_distance)*100):2.2f}%") ## 100m 반경에 이디야 매장이 있는 스타벅스 매장의 수
print(f"1000m\t 안에 스타벅스 매장이 없는 비율 : \t{sum(df_ediya_w_distance['count_1000m']==0)}개, \t{(sum(df_ediya_w_distance['count_1000m']==0)/len(df_ediya_w_distance)*100):2.2f}%") ## 100m 반경에 이디야 매장이 있는 스타벅스 매장의 수

이디야 통계 (근처 스타벅스 매장이 없는 경우)
100m	 안에 스타벅스 매장이 없는 비율 : 	596개, 	83.01%
200m	 안에 스타벅스 매장이 없는 비율 : 	447개, 	62.26%
500m	 안에 스타벅스 매장이 없는 비율 : 	212개, 	29.53%
1000m	 안에 스타벅스 매장이 없는 비율 : 	45개, 	6.27%


# 결론
---
1. 스타벅스 매장의 500m 안에, 이디야가 있는 비율은 91.84% 이나, 이디야 매장의 500m 안에, 스타벅스 매장이 없는 비율은 29.53%로 상대적으로 낮지않다. 의도적으로 가까이 배치했을 수 있으나, 스타벅스가 없는 장소에도 충분히 이디야가 진출했다고 볼 수 있다. 따라서 이디야가 커버하는 커버리지가 더 넓어서 보이는 요소로 볼 수 있으므로, 항상 참이라고 보긴 어렵다.
2. 그러면 스타벅스에 있는 장소에 우선적으로 이디야가 진출하고, 그 외의 장소에도 진출했을까?
전체 스타벅스 주변에 있는 이디야 커피 매장이 서울 전체의 면적의 커버리지를 비교해보자
